In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [2]:
#Preparing data
housing = fetch_california_housing()
house_X, y_house = housing.data, housing.target
m,n = house_X.shape
SS = StandardScaler()
X_scaled = SS.fit_transform(house_X)
X_scaled_B = np.c_[np.ones((m, 1)), X_scaled]

In [12]:
tf.reset_default_graph()
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_dzienniki"
logdir = "{}/przebieg-{}".format(root_logdir, now)

def fetch_batch(epoch, batch_index, batch_size):
#     np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = X_scaled_B[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch


X = tf.placeholder(dtype = tf.float32 ,name = 'X')
y = tf.placeholder(dtype = tf.float32 ,name = 'y')
theta = tf.Variable(tf.random_uniform([n+1, 1],-1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predicted_vals')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(mse)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

saver = tf.train.Saver()
init = tf.global_variables_initializer()



batch_size = 100
n_epochs = 100
n_batches = int(np.ceil(m / batch_size))
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict = {X:X_batch,y:y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict = {X:X_batch,y:y_batch})
    best_theta = theta.eval()
    file_writer.close()
print(best_theta)


[[  2.05716515e+00]
 [  8.29733849e-01]
 [  1.13596402e-01]
 [ -2.67577291e-01]
 [  3.10342550e-01]
 [  1.29029620e-03]
 [ -3.31210233e-02]
 [ -8.99932742e-01]
 [ -8.51529181e-01]]


In [16]:
tf.reset_default_graph()
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_dzienniki"
logdir = "{}/przebiegGrad-{}".format(root_logdir, now)


X = tf.constant(X_scaled_B, dtype = tf.float32 ,name = 'X')
y = tf.constant(y_house.reshape(-1,1),dtype = tf.float32 ,name = 'y')
theta = tf.Variable(tf.random_uniform([n+1, 1],-1.0,1.0), name = 'theta')

y_pred = tf.matmul(X, theta, name = 'predicted_vals')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = 'mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - 0.01*gradients)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(20000):
        if i % 200 == 0:
            summary_str = mse_summary.eval()
            file_writer.add_summary(summary_str, i)
        sess.run(training_op)
    best_theta = theta.eval()
    file_writer.close()
print(best_theta)

[[ 2.06855249]
 [ 0.82963288]
 [ 0.11875445]
 [-0.26554993]
 [ 0.30571496]
 [-0.00450218]
 [-0.03932685]
 [-0.89985341]
 [-0.87051022]]
